# Importación de Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
SEQ_LEN = 60
BATCH_SIZE = 64

# Uso de "Device"

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carga de datos

In [3]:
data = pd.read_csv('btcusd_1-min_data.csv', nrows=526000)
data.head()
data.count()

Timestamp    526000
Open         526000
High         526000
Low          526000
Close        526000
Volume       526000
dtype: int64

# Preprocesamiento de datos

In [4]:
# Normalizar los datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])

# Crear secuencias de tiempo de entrada y salida
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length][3]  # Precio de cierre
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

#  Crear secuencias de tiempo de entrada y salida
X, y = create_sequences(data_scaled, SEQ_LEN)

# Convertir a tensores
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Crear el DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, BATCH_SIZE, shuffle=True)

# Definición de Modelo

In [5]:
class BitcoinLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BitcoinLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_size = 5  # Número de características (Open, High, Low, Close, Volume)
hidden_size = 50
num_layers = 4
output_size = 1  # Predicción del precio de cierre

model = BitcoinLSTM(input_size, hidden_size, num_layers, output_size).to(device)

# Entrenamiento del Modelo

In [6]:
# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el modelo
num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/home/silva/.local/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
